In [6]:
import cvxpy as cp
import numpy as np

Traffic Network (AM peak)

In [7]:
# Variables
t_1, theta_1, gamma_1 = 7, 0.160, 2.85
t_2, theta_2, gamma_2 = 10, 0.308, 2.63
t_3, theta_3, gamma_3 = 5, 0.241, 2.43
t_4, theta_4, gamma_4 = 7, 0.176, 2.57
x = cp.Variable()

# Objective function
objective = cp.sum([(t_2*(x + x**2*cp.inv_pos(1+cp.exp(-(1/(theta_2*gamma_2)))))),
                   (t_3*(x + x**2*cp.inv_pos(1+cp.exp(-(1/(theta_3*gamma_3)))))),
                   (t_4*((1-x) + (1-2*x+x**2)*cp.inv_pos(1+cp.exp(-(1/(theta_4*gamma_4)))))),
                   (t_1*((1-x) + (1-2*x+x**2)*cp.inv_pos(1+cp.exp(-(1/(theta_1*gamma_1))))))
                    ])

# Construct the problem
objective = cp.Minimize(objective)
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# Optimal objective value is returned by `prob.solve()`
result = prob.solve()
print("Optimal proportion of users travel to top path is {x_1} and to bottom path is {x_2}".format(
    x_1=np.round(x.value, 3), x_2=np.round(1-x.value, 3)))
print("Total Delay is {result}".format(result=np.round(result, 3)))

print(theta_1*gamma_1,
      theta_2*gamma_2,
      theta_3*gamma_3,
      theta_4*gamma_4)

Optimal proportion of users travel to top path is 0.492 and to bottom path is 0.508
Total Delay is 20.645
0.456 0.81004 0.58563 0.45231999999999994


In [8]:
# Variables
t_1, theta_1, gamma_1 = 7, 0.160, 2.85
t_2, theta_2, gamma_2 = 10, 0.308, 2.63
t_3, theta_3, gamma_3 = 5, 0.241, 2.43
t_4, theta_4, gamma_4 = 7, 0.176, 2.57
t_5, theta_5, gamma_5 = 3, 0.216, 3
t_6, theta_6, gamma_6 = 3, 0.222, 3

x_1 = cp.Variable()
x_2 = cp.Variable()
x_3 = cp.Variable()
x_4 = cp.Variable()
x_5 = cp.Variable()
x_6 = cp.Variable()

# Objective function
objective = cp.sum([(t_2*(x_2 + x_2**2*cp.inv_pos(1+cp.exp(-(1/(theta_2*gamma_2)))))),
                   (t_1*(x_1 + x_1**2*cp.inv_pos(1+cp.exp(-(1/(theta_1*gamma_1)))))),
                   (t_5*(x_5 + x_5**2*cp.inv_pos(1+cp.exp(-(1/(theta_5*gamma_5)))))),
                   (t_6*(x_6 + x_6**2*cp.inv_pos(1+cp.exp(-(1/(theta_6*gamma_6)))))),
                   (t_3*(x_3 + x_3**2*cp.inv_pos(1+cp.exp(-(1/(theta_3*gamma_3)))))),
                   (t_4*(x_4 + x_4**2*cp.inv_pos(1+cp.exp(-(1/(theta_4*gamma_4)))))),
                    ])

# Construct the problem
objective = cp.Minimize(objective)
constraints = [x_1+x_2 == 1, x_3+x_4 == 1, x_3 == x_2+x_5-x_6, x_4 == x_1+x_6-x_5,
               0 <= x_1, 0 <= x_2, 0 <= x_3, 0 <= x_4, 0 <= x_5, 0 <= x_6, x_1 <= 1, x_2 <= 1, x_3 <= 1, x_4 <= 1, x_5 <= 1, x_6 <= 1]
prob = cp.Problem(objective, constraints)

# Optimal objective value is returned by `prob.solve()`
result = prob.solve()

print("Total Delay is {result}".format(result=np.round(result, 3)))

print(theta_1*gamma_1,
      theta_2*gamma_2,
      theta_3*gamma_3,
      theta_4*gamma_4)

print(x_1.value, x_2.value, x_3.value, x_4.value, x_5.value, x_6.value)

Total Delay is 20.6
0.456 0.81004 0.58563 0.45231999999999994
0.5387759190480229 0.46122408095197703 0.5339750864627306 0.4660249135372693 0.07275100551075364 -5.1988207427930004e-23


In [9]:
import pandas as pd
# empty dataframe with 6 columns: top_path, bottom_path, cycling, delay_driving, delay_cycling, delay_total
df = pd.DataFrame(columns=['top_path', 'bottom_path', 'cycling', 'delay_driving', 'delay_cycling', 'delay_total'])

for i in np.arange(0.1, 1, 0.1):
    # Variables
    t_1, theta_1, gamma_1 = 7, 0.160, (-0.625+2.85)
    t_2, theta_2, gamma_2 = 10, 0.308, (-0.625+2.63)
    t_3, theta_3, gamma_3=5, 0.241, (-0.625+2.43)
    t_4, theta_4, gamma_4=7, 0.176, (-0.625+2.57)
    x_1 = cp.Variable()
    x_2 = cp.Variable()

    # Objective function
    objective = cp.sum([(t_2*(x_1 + x_1**2*cp.inv_pos(1+cp.exp(-(1/(theta_2*gamma_2)))))),
                    (t_3*(x_1 + x_1**2*cp.inv_pos(1+cp.exp(-(1/(theta_3*gamma_3)))))),
                    (t_4*(x_2 + x_2**2*cp.inv_pos(1+cp.exp(-(1/(theta_4*gamma_4)))))),
                    (t_1*(x_2 + x_2**2)*cp.inv_pos(1+cp.exp(-(1/(theta_1*gamma_1)))))
                    ])

    # Construct the problem
    objective = cp.Minimize(objective)
    constraints = [0<=x_1, x_1+x_2 == 1-i, 0<=x_2]
    prob = cp.Problem(objective, constraints)

    # Optimal objective value is returned by `prob.solve()`
    result = prob.solve()
    # add row to dataframe
    df.loc[len(df)] = [np.round(x_2.value,3), np.round(x_1.value,3), i, np.round(result,3), 34*i, np.round(result+34*i,3)]
#save dataframe to csv index false
df.to_csv('dataframe_1.csv', index=False)
df


,top_path,bottom_path,cycling,delay_driving,delay_cycling,delay_total
0,0.471,0.429,0.1,18.152,3.4,21.552
1,0.421,0.379,0.2,15.611,6.8,22.411
2,0.372,0.328,0.3,13.200,10.2,23.400
3,0.323,0.277,0.4,10.919,13.6,24.519
4,0.273,0.227,0.5,8.770,17.0,25.770
5,0.224,0.176,0.6,6.751,20.4,27.151
6,0.175,0.125,0.7,4.862,23.8,28.662
7,0.125,0.075,0.8,3.105,27.2,30.305
8,0.076,0.024,0.9,1.478,30.6,32.078


Traffic Network (PM-non peak)

In [10]:
# Variables
t_1, theta_1, gamma_1 = 7, 0.153, 2.85
t_2, theta_2, gamma_2 = 10, 0.249, 2.63
t_3, theta_3, gamma_3 = 5, 0.203, 2.43
t_4, theta_4, gamma_4 = 7, 0.137, 2.57
x = cp.Variable()

# Objective function
objective = cp.sum([(t_2*(x + x**2*cp.inv_pos(1+cp.exp(-(1/(theta_2*gamma_2)))))),
                   (t_3*(x + x**2*cp.inv_pos(1+cp.exp(-(1/(theta_3*gamma_3)))))),
                   (t_4*((1-x) + (1-2*x+x**2)*cp.inv_pos(1+cp.exp(-(1/(theta_4*gamma_4)))))),
                   (t_1*((1-x) + (1-2*x+x**2)*cp.inv_pos(1+cp.exp(-(1/(theta_1*gamma_1))))))
                    ])
                    
# Construct the problem
objective = cp.Minimize(objective)
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# Optimal objective value is returned by `prob.solve()`
result = prob.solve()
print("Optimal proportion of users travel to top path is {x_1} and to bottom path is {x_2}".format(
    x_1=np.round(x.value, 3), x_2=np.round(1-x.value, 3)))
print("Total Delay is {result}".format(result=np.round(result, 3)))

print(theta_1*gamma_1,
      theta_2*gamma_2,
      theta_3*gamma_3,
      theta_4*gamma_4)

Optimal proportion of users travel to top path is 0.487 and to bottom path is 0.513
Total Delay is 20.897
0.43605 0.65487 0.49329000000000006 0.35209


In [11]:
# Variables
t_1, theta_1, gamma_1 = 7, 0.153, 2.85
t_2, theta_2, gamma_2 = 10, 0.249, 2.63
t_3, theta_3, gamma_3 = 5, 0.203, 2.43
t_4, theta_4, gamma_4 = 7, 0.137, 2.57
t_5, theta_5, gamma_5 = 3, 0.142, 3
t_6, theta_6, gamma_6 = 3, 0.136, 3

x_1 = cp.Variable()
x_2 = cp.Variable()
x_3 = cp.Variable()
x_4 = cp.Variable()
x_5 = cp.Variable()
x_6 = cp.Variable()

# Objective function
objective = cp.sum([(t_2*(x_2 + x_2**2*cp.inv_pos(1+cp.exp(-(1/(theta_2*gamma_2)))))),
                   (t_1*(x_1 + x_1**2*cp.inv_pos(1+cp.exp(-(1/(theta_1*gamma_1)))))),
                   (t_5*(x_5 + x_5**2*cp.inv_pos(1+cp.exp(-(1/(theta_5*gamma_5)))))),
                   (t_6*(x_6 + x_6**2*cp.inv_pos(1+cp.exp(-(1/(theta_6*gamma_6)))))),
                   (t_3*(x_3 + x_3**2*cp.inv_pos(1+cp.exp(-(1/(theta_3*gamma_3)))))),
                   (t_4*(x_4 + x_4**2*cp.inv_pos(1+cp.exp(-(1/(theta_4*gamma_4)))))),
                    ])

# Construct the problem
objective = cp.Minimize(objective)
constraints = [x_1+x_2 == 1, x_3+x_4 == 1, x_3 == x_2+x_5-x_6, x_4 == x_1+x_6-x_5,
               0 <= x_1, 0 <= x_2, 0 <= x_3, 0 <= x_4, 0 <= x_5, 0 <= x_6, x_1 <= 1, x_2 <= 1, x_3 <= 1, x_4 <= 1, x_5 <= 1, x_6 <= 1]
prob = cp.Problem(objective, constraints)

# Optimal objective value is returned by `prob.solve()`
result = prob.solve()

print("Total Delay is {result}".format(result=np.round(result, 3)))

print(theta_1*gamma_1,
      theta_2*gamma_2,
      theta_3*gamma_3,
      theta_4*gamma_4)

print(x_1.value, x_2.value, x_3.value, x_4.value, x_5.value, x_6.value)


Total Delay is 20.836
0.43605 0.65487 0.49329000000000006 0.35209
0.548298686797102 0.4517013132028979 0.5338069568871576 0.4661930431128425 0.08210564368425952 -6.476616623589546e-23


In [14]:
import pandas as pd
# empty dataframe with 6 columns: top_path, bottom_path, cycling, delay_driving, delay_cycling, delay_total
df = pd.DataFrame(columns=['top_path', 'bottom_path', 'cycling',
                  'delay_driving', 'delay_cycling', 'delay_total'])

for i in np.arange(0.1, 1, 0.1):
    # Variables
    t_1, theta_1, gamma_1 = 7, 0.153, (-0.625+2.85)
    t_2, theta_2, gamma_2 = 10, 0.249, (-0.625+2.63)
    t_3, theta_3, gamma_3 = 5, 0.203, (-0.625+2.43)
    t_4, theta_4, gamma_4 = 7, 0.137, (-0.625+2.57)
    x_1 = cp.Variable()
    x_2 = cp.Variable()

    # Objective function
    objective = cp.sum([(t_2*(x_1 + x_1**2*cp.inv_pos(1+cp.exp(-(1/(theta_2*gamma_2)))))),
                        (t_3*(x_1 + x_1**2*cp.inv_pos(1+cp.exp(-(1/(theta_3*gamma_3)))))),
                        (t_4*(x_2 + x_2**2*cp.inv_pos(1+cp.exp(-(1/(theta_4*gamma_4)))))),
                        (t_1*(x_2 + x_2**2)*cp.inv_pos(1 +
                         cp.exp(-(1/(theta_1*gamma_1)))))
                        ])

    # Construct the problem
    objective = cp.Minimize(objective)
    constraints = [0 <= x_1, x_1+x_2 == 1-i, 0 <= x_2]
    prob = cp.Problem(objective, constraints)

    # Optimal objective value is returned by `prob.solve()`
    result = prob.solve()
    # add row to dataframe
    df.loc[len(df)] = [np.round(x_2.value, 3), np.round(x_1.value, 3),
                       i, np.round(result, 3), 34*i, np.round(result+34*i, 3)]
df.to_csv('dataframe_2.csv', index=False)
df

,top_path,bottom_path,cycling,delay_driving,delay_cycling,delay_total
0,0.475,0.425,0.1,18.340,3.4,21.740
1,0.425,0.375,0.2,15.761,6.8,22.561
2,0.375,0.325,0.3,13.316,10.2,23.516
3,0.325,0.275,0.4,11.006,13.6,24.606
4,0.275,0.225,0.5,8.832,17.0,25.832
5,0.225,0.175,0.6,6.792,20.4,27.192
6,0.175,0.125,0.7,4.887,23.8,28.687
7,0.125,0.075,0.8,3.118,27.2,30.318
8,0.075,0.025,0.9,1.483,30.6,32.083
